In [1]:
import re
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import os

In [2]:
def convert_pdf_to_txt(outfile, infile):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    
    pagenos = set()
    maxpages = 0
    # output option
    imagewriter = None
    rotation = 0
    stripcontrol = False
    layoutmode = 'normal'
    encoding = 'utf-8'
    #pageno = 1
    pagenos=set()
    scale = 1
    caching = True
    showpageno = True
    laparams = LAParams()
    

    outfp = open(outfile, 'w', encoding=encoding)
    
    device = TextConverter(rsrcmgr, outfp, laparams=laparams,
                               imagewriter=imagewriter)
        
    password = b''
    maxpages = 0
    caching = True
    
    with open(infile, 'rb') as fp:
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.get_pages(fp, pagenos,
                                          maxpages=maxpages, password=password,
                                          caching=caching, check_extractable=True):
            #page.rotate = (page.rotate+rotation) % 360
            interpreter.process_page(page)
            
    device.close()
    outfp.close()
    
    #text = retstr.getvalue()
    

In [ ]:
data_dir = '/Users/atkumar/Downloads/fwdwatsonproject'

all_head = []
substitution = None
for f in os.listdir(data_dir):
    full_path = os.path.join(data_dir, f)
    if os.path.isfile(full_path) and f.endswith('.pdf'):
        print(full_path)
        txt_filename = os.path.join(data_dir, os.path.splitext(f)[0] + '.txt')
        convert_pdf_to_txt(txt_filename, full_path)
       

In [4]:
def filter_doc(text_filename):
    with open(text_filename) as f:
        text_doc = f.read().splitlines()
    
    text_doc=[i.replace('\xe2\x80\x9c','') for i in text_doc]
    text_doc=[i.replace('\xe2\x80\x9d','') for i in text_doc ]
    text_doc=[i.replace('\xe2\x80\x99s','') for i in text_doc ]

    text_doc = [x for x in text_doc if x != ' ']
    text_doc = [x for x in text_doc if x != '']

    text_doc=[re.sub("[^a-zA-Z]+", " ", s) for s in text_doc]
    return text_doc

In [5]:
all_text = []
for f in os.listdir(data_dir):
    full_path = os.path.join(data_dir, f)
    if os.path.isfile(full_path) and f.endswith('.txt'):
        print(full_path)
        all_text.extend(filter_doc(full_path))

/Users/atkumar/Downloads/fwdwatsonproject/Cairn CLO IX - Preliminary OC (22Jan2018).txt
/Users/atkumar/Downloads/fwdwatsonproject/As printed_Neuberger Berman 27 - Preliminary Offering Circular.txt
/Users/atkumar/Downloads/fwdwatsonproject/OCP 2014-5 Refi - First Preliminary Offering Circular (as-printed).txt
/Users/atkumar/Downloads/fwdwatsonproject/Marathon CLO XI - Preliminary Offering Circular dated January 10, 2018 f....txt
/Users/atkumar/Downloads/fwdwatsonproject/Voya CLO 2017-4 -  Offering Circular dated Nov. 22, 2017.txt
/Users/atkumar/Downloads/fwdwatsonproject/Dryden 59 - First Preliminary Prospectus 01.02.18.txt
/Users/atkumar/Downloads/fwdwatsonproject/Change-Pro Redline - Magnetite VII Refinancing - Conformed Indenture (20....txt
/Users/atkumar/Downloads/fwdwatsonproject/MS - CLOMARKET_20180131_WATSON INDENTURE.txt
/Users/atkumar/Downloads/fwdwatsonproject/Voya 2016-1 - Indenture (Reset).txt
/Users/atkumar/Downloads/fwdwatsonproject/Voya 2016-1 - Final Offering Circular (R

In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import numpy as np
%pylab
%matplotlib inline
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


/Users/atkumar/anaconda3/envs/neural_ner/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [7]:
vect=CountVectorizer(ngram_range=(1,1),stop_words='english')
dtm=vect.fit_transform(all_text)
pd.DataFrame(dtm.toarray(),columns=vect.get_feature_names())
lda=LatentDirichletAllocation(n_components=5)
lda_dtf = lda.fit_transform(dtm)

In [8]:
import numpy as np
sorting=np.argsort(lda.components_)[:,::-1]
features=np.array(vect.get_feature_names())

In [9]:
import mglearn
mglearn.tools.print_topics(topics=range(5), feature_names=features,
sorting=sorting, topics_per_chunk=5, n_words=10)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
issuer        manager       notes         date          collateral    
trustee       investment    class         payment       obligation    
section       issuer        subordinated  account       obligations   
tax           collateral    secured       period        rating        
shall         securities    note          proceeds      rate          
indenture     retention     holders       amounts       principal     
income        agreement     global        redemption    debt          
holder        management    respect       prior         moody         
information   portfolio     refinancing   principal     credit        
person        act           additional    payments      issuer        




In [20]:
#top 4 sentences from topic 2
topic_id = 3
event_time_Topic=np.argsort(lda_dtf[:,topic_id])[::-1]
for i in event_time_Topic[:4]:
    print(".".join(all_text[i].split(".")[:2]) + ".\n")
    print('---------------------')

Up Call Redemption on the day preceding the Redemption Date and y in the case of a Refinancing on the day immediately preceding the Redemption Date provided further that with respect to any Payment Date and any amounts payable to the Issuer under a Hedge Agreement the Collection Period will commence on the day after the prior Payment Date and end on such Payment Date provided further that any Refinancing Proceeds received on the Redemption Date shall be deemed to have been received on the day immediately preceding the Redemption Date .

---------------------
 C to the payment pro rata based on amounts due of any amounts due to a Hedge Counterparty under a Hedge Agreement other than amounts due as a result of the termination or partial termination of such Hedge Agreement and any amounts due to a Hedge Counterparty under a Hedge Agreement pursuant to an early termination or partial termination of such Hedge Agreement as a result of a Priority Hedge Termination Event .

------------------

In [10]:
topic_id = 3
Domain_Name_Topic=np.argsort(lda_dtf[:,topic_id])[::-1]
for i in Domain_Name_Topic[:4]:
    print(".".join(all_text[i].split(".")[:2]) + ".\n")
    print('---------------------')

Up Call Redemption on the day preceding the Redemption Date and y in the case of a Refinancing on the day immediately preceding the Redemption Date provided further that with respect to any Payment Date and any amounts payable to the Issuer under a Hedge Agreement the Collection Period will commence on the day after the prior Payment Date and end on such Payment Date provided further that any Refinancing Proceeds received on the Redemption Date shall be deemed to have been received on the day immediately preceding the Redemption Date .

---------------------
 C to the payment pro rata based on amounts due of any amounts due to a Hedge Counterparty under a Hedge Agreement other than amounts due as a result of the termination or partial termination of such Hedge Agreement and any amounts due to a Hedge Counterparty under a Hedge Agreement pursuant to an early termination or partial termination of such Hedge Agreement as a result of a Priority Hedge Termination Event .

------------------

In [19]:
from __future__ import  print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
zit=pyLDAvis.sklearn.prepare(lda,dtm,vect)
pyLDAvis.show(zit)